<a href="https://colab.research.google.com/github/kapamawi/AI/blob/main/2_3_2___kalkulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install -q llama-index-llms-openai llama-index

Kod instaluje dwie biblioteki Pythona przy użyciu menedżera pakietów pip:

1. llama-index-llms-openai - zawiera interfejsy do komunikacji z modelami językowymi OpenAI
2. llama-index - framework do budowania aplikacji wykorzystujących duże modele językowe

Flaga -q (quiet) powoduje, że pip wyświetla tylko najważniejsze informacje podczas instalacji, pomijając szczegółowe logi procesu.

Te biblioteki razem pozwalają na tworzenie aplikacji wykorzystujących modele językowe OpenAI do przetwarzania i generowania tekstu, a także indeksowania i wyszukiwania w dużych zbiorach danych tekstowych.

In [ ]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
from google.colab import userdata
import os

Ten kod importuje niezbędne elementy do stworzenia agenta AI oraz konfiguracji dostępu do API:

`from llama_index.core.agent import ReActAgent` - importuje klasę ReActAgent, która implementuje agenta AI wykorzystującego podejście ReAct (Reasoning and Acting)

`from llama_index.llms.openai import OpenAI` - importuje klasę do komunikacji z modelami OpenAI

`from llama_index.core.llms import ChatMessage` - importuje klasę reprezentującą wiadomości w konwersacji

`from llama_index.core.tools import BaseTool, FunctionTool` - importuje narzędzia pozwalające agentowi wykonywać konkretne zadania:
- BaseTool to bazowa klasa dla wszystkich narzędzi
- FunctionTool pozwala przekształcić zwykłe funkcje w narzędzia dla agenta

`from google.colab import userdata` i `import os` - umożliwiają dostęp do danych użytkownika w Google Colab oraz operacje na systemie plików

In [ ]:
class CFG:
  model = "gpt-4o-mini"

In [ ]:
os.environ['OPENAI_API_KEY'] = userdata.get('openaivision')

# Functions


In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

Ten fragment kodu definiuje prostą funkcję mnożenia i przekształca ją w narzędzie, które będzie mógł używać agent AI:

`def multiply(a: int, b: int) -> int:` - definicja funkcji z adnotacjami typów:
- przyjmuje dwa argumenty typu całkowitego (int)
- zwraca wynik również jako liczbę całkowitą
- komentarz w potrójnych cudzysłowach (docstring) opisuje działanie funkcji

`multiply_tool = FunctionTool.from_defaults(fn=multiply)` - tworzy narzędzie z funkcji multiply:
- FunctionTool.from_defaults() automatycznie tworzy narzędzie na podstawie podanej funkcji
- narzędzie będzie dostępne dla agenta do wykonywania operacji mnożenia

Agent będzie mógł użyć tego narzędzia, gdy będzie potrzebował pomnożyć dwie liczby w trakcie wykonywania swoich zadań.

In [ ]:
def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

Ten fragment jest analogiczny do poprzedniego, ale implementuje dodawanie zamiast mnożenia:

`def add(a: int, b: int) -> int:` - definicja funkcji dodawania:
- przyjmuje dwa argumenty całkowite
- zwraca ich sumę jako liczbę całkowitą
- docstring opisuje funkcjonalność

`add_tool = FunctionTool.from_defaults(fn=add)` - przekształca funkcję add w narzędzie:
- tworzy narzędzie, które agent będzie mógł wykorzystać do dodawania liczb
- używa tej samej metody FunctionTool.from_defaults() co poprzednio

W ten sposób agent otrzymuje dostęp do dwóch podstawowych operacji arytmetycznych: dodawania i mnożenia.

# Query


In [ ]:
llm = OpenAI(model= CFG.model)

Ta linia tworzy instancję modelu językowego:

`llm = OpenAI(model=CFG.model)` - inicjalizuje model OpenAI:
- używa modelu zdefiniowanego wcześniej w klasie CFG (gpt-4o-mini)
- tworzy obiekt llm (language learning model), który będzie odpowiedzialny za przetwarzanie języka naturalnego
- ten obiekt będzie używany przez agenta do rozumienia poleceń i generowania odpowiedzi

Jest to kluczowy element konfiguracji, który łączy nasz kod z API OpenAI i umożliwia korzystanie z możliwości modelu językowego.

In [ ]:
agent = ReActAgent.from_tools([multiply_tool, add_tool], llm=llm, verbose=True)

Ta linia tworzy agenta AI z określonymi narzędziami i modelem językowym:

`ReActAgent.from_tools()` - tworzy nowego agenta wykorzystującego metodologię ReAct (Reasoning and Acting):
- `[multiply_tool, add_tool]` - lista narzędzi dostępnych dla agenta (w tym przypadku mnożenie i dodawanie)
- `llm=llm` - przypisuje wcześniej skonfigurowany model językowy
- `verbose=True` - włącza szczegółowe logowanie, dzięki czemu będziemy widzieć proces rozumowania agenta

Ten agent będzie mógł:
- interpretować polecenia w języku naturalnym
- decydować, które narzędzie użyć
- wykonywać operacje matematyczne
- przedstawiać wyniki swoich działań

In [ ]:
prompt_dict = agent.get_prompts()
for k, v in prompt_dict.items():
    print(f"Prompt: {k}\n\nValue: {v.template}")

Prompt: agent_worker:system_prompt

Value: You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

NEVER surround your response with markdown code markers. You may

Ten kod wyświetla szablony promptów używanych przez agenta:

`prompt_dict = agent.get_prompts()` - pobiera słownik zawierający wszystkie szablony promptów agenta

Pętla `for k, v in prompt_dict.items():` przechodzi przez wszystkie elementy słownika:
- `k` to klucz (nazwa promptu)
- `v` to obiekt promptu
- `v.template` zawiera faktyczny tekst szablonu
- `print(f"Prompt: {k}\n\nValue: {v.template}")` wyświetla nazwę każdego promptu i jego zawartość

Dzięki temu możemy zobaczyć, jakie szablony instrukcji agent wykorzystuje do:
- planowania działań
- podejmowania decyzji
- formatowania odpowiedzi
- interakcji z narzędziami

In [ ]:
response = agent.chat("Jaki jest wynik 20+(2*4)? Przelicz krok po kroku ")

> Running step e0ce578b-c8ed-4411-b52b-372594c02600. Step input: Jaki jest wynik 20+(2*4)? Przelicz krok po kroku 
Thought: Użytkownik prosi o obliczenie wyrażenia 20 + (2 * 4) krok po kroku. Najpierw obliczę część mnożenia, a następnie dodam wynik do 20.
Action: multiply
Action Input: {'a': 2, 'b': 4}
Observation: 8
> Running step 6228fed3-7bfd-410c-bae1-e5bd25a940fe. Step input: None
Thought: Wynik mnożenia 2 * 4 to 8. Teraz dodam ten wynik do 20.
Action: add
Action Input: {'a': 20, 'b': 8}
Observation: 28
> Running step 7260049b-02b9-4eea-a665-20070e0a9108. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Wynik wyrażenia 20 + (2 * 4) to 28.


In [ ]:
response = agent.chat("What is the result of 20+(2*4)? Calculate step by step")

> Running step a0f09c69-d4b4-4099-a1bc-3a52e0af2ab6. Step input: What is the result of 20+(2*4)? Calculate step by step
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 2, 'b': 4}
Observation: 8
> Running step a6074bba-5126-4aea-bd22-191e83e31647. Step input: None
Thought: Now that I have the result of the multiplication, I can proceed to add it to 20.
Action: add
Action Input: {'a': 20, 'b': 8}
Observation: 28
> Running step 9e8d1f01-ec72-4804-abba-fd3eba534d59. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The result of 20 + (2 * 4) is 28.
